In [ ]:
pip install pandas matplotlib folium


In [ ]:
import pandas as pd

# 🔽 Load the downloaded dataset
df = pd.read_csv("modis_2024_India.csv")

# 🔍 Show first few rows
print(df.head())

# 🧾 Basic info
print("\nColumns:", df.columns)
print("\nSummary:\n", df.describe())

# 🔍 Unique satellites and confidence levels
print("\nSatellites:", df['satellite'].unique())
print("Confidence levels:", df['confidence'].unique())


# Forest Fire Detection in India using MODIS FIRMS Data

This notebook focuses on analyzing active fire data over India using satellite-based thermal anomaly detection from NASA's MODIS sensor. The dataset includes information on fire hotspots, brightness, confidence levels, and satellite metadata.

---

## Dataset Description

Each row in the dataset represents a fire detection (thermal anomaly) observed by a satellite. Below are the key fields included:

###  latitude
- Latitude where the fire was detected (between 8°N to 37°N for India)

###  longitude
- Longitude where the fire was detected (between 68°E to 97°E)

###  brightness
- Brightness temperature (Kelvin) measured by the satellite in the fire pixel

###  scan & track
- Dimensions (in kilometers) of the satellite sensor's scan pixel

###  acq_date
- Date when the fire was detected (`YYYY-MM-DD` format)

###  acq_time
- Time of detection in UTC (`HHMM` format)

###  satellite
- Satellite that detected the fire: `T` for Terra or `A` for Aqua

###  instrument
- Usually `MODIS` (Moderate Resolution Imaging Spectroradiometer)

###  confidence
- Confidence level of the detection:
  - Can be a percentage (0–100) or label like `'low'`, `'nominal'`, `'high'`

###  version
- Product version (e.g., `6.1`)

###  bright_t31
- Brightness temperature of the background (non-fire) pixel in Kelvin

###  frp
- Fire Radiative Power (intensity of the fire) in megawatts (MW)

###  daynight
- Indicates whether the fire was detected during the `day` or `night`

###  type
- Indicates the type of thermal source:
  - `0` - Presumed vegetation fire
  - `1` - Active volcano
  - `2` - Other (e.g., industrial heat source)

---

##  Focus Area

This analysis is limited to **India**, filtered using:
- Latitude: `8°N to 37°N`
- Longitude: `68°E to 97°E`

---

##  Objectives of This Notebook

- Load and explore MODIS FIRMS fire data
- Filter and visualize India-specific fire detections
- Analyze fire distribution over date, region, and confidence
- Optionally simulate fire spread or build predictive ML models

---

##  Data Source

- **NASA FIRMS (Fire Information for Resource Management System)**  
  🔗 [https://firms.modaps.eosdis.nasa.gov/](https://firms.modaps.eosdis.nasa.gov/)

---




In [ ]:
df

In [ ]:

# Create a folium map centered on a mean location
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=4)

# Add fire points
for _, row in df.iterrows():
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=3,
        color='red' if row['confidence'] >= 80 else 'orange',
        fill=True,
        fill_opacity=0.7,
        popup=f"Date: {row['acq_date']} | Brightness: {row['brightness']}"
    ).add_to(m)

# Show map
m


In [ ]:

# Create a folium map centered on a mean location
m = folium.Map(location=[df['latitude'].mean(), df['longitude'].mean()], zoom_start=4)

# Add fire points
for _, row in df.iterrows():
    folium.CircleMarker(
        location=(row['latitude'], row['longitude']),
        radius=3,
        color='red' if row['confidence'] >= 80 else 'orange',
        fill=True,
        fill_opacity=0.7,
        popup=f"Date: {row['acq_date']} | Brightness: {row['brightness']}"
    ).add_to(m)

# Show map
m

In [ ]:
Analyze by Time of Day (Day vs Night)

In [ ]:
import matplotlib.pyplot as plt

# Count by day/night
df['daynight'].value_counts().plot(
    kind='pie',
    autopct='%1.1f%%',
    colors=['gold', 'skyblue'],
    title=" Fire Detections: Day vs Night"
)

plt.ylabel('')  # remove y-label
plt.show()


In [ ]:
plt.figure(figsize=(8,5))
plt.hist(df['brightness'], bins=30, color='darkred', edgecolor='black')
plt.title(" Brightness Distribution of Fire Pixels")
plt.xlabel("Brightness (Kelvin)")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

In [ ]:
# Drop any missing values
df_clean = df.dropna()

# Select relevant features for ML
features = df_clean[['latitude', 'longitude', 'brightness', 'scan', 'track', 'bright_t31', 'frp']]
target = df_clean['confidence']  # or convert to classification: high vs low

print(features.head())

In [ ]:
# Convert confidence to binary classification
target_class = df_clean['confidence'].apply(lambda x: 1 if x >= 80 else 0)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target_class, test_size=0.2, random_state=42)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


In [ ]:
from sklearn.metrics import classification_report, accuracy_score

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


In [ ]:
import matplotlib.pyplot as plt

importances = model.feature_importances_
feature_names = features.columns

plt.figure(figsize=(8,5))
plt.barh(feature_names, importances, color='teal')
plt.xlabel("Importance")
plt.title("Feature Importance for Fire Confidence Classification")
plt.grid(True)
plt.tight_layout()
plt.show()
